# WeRateDogs Data Wrangling Project

## Gather

In [86]:
import pandas as pd
import requests
import tweepy
import json

### Read in Enhanced Twitter Archive Data

In [87]:
twit_ar_df = pd.read_csv('../data/twitter-archive-enhanced.csv')

### Read in Image Prediction TSV using Requests

In [88]:
resp = requests.get('https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv')

In [89]:
with open('../data/image-predictions.tsv', 'wb') as out_file:
    out_file.write(resp.content)

### Read in Retweet Amount and Like Count Using Tweepy

In [90]:
import tweepy

In [91]:
consumer_key='mSWxjuVU1Vr5joR56Ncb4Iyk8'
consumer_secret='REPmOAzwr9UlsKnHvdNmjFMriBpQcEq8aClmebIuFdd40TNgKk'

access_token='1055804373467623425-EBOwt17JikaJdFPr2Bi9KUNT4Xrm7k'
access_token_secret='0yv3cowB6jqhcQFCpZOcoit2OBfYkSkAU7Ph1HIAd8EFJ'

In [92]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

api = tweepy.API(auth_handler=auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

In [93]:
tweet_id = '892420643555336193'
tweet = api.get_status(tweet_id, tweet_mode='extended')
tweet.__dict__.keys()

dict_keys(['_api', '_json', 'created_at', 'id', 'id_str', 'full_text', 'truncated', 'display_text_range', 'entities', 'extended_entities', 'source', 'source_url', 'in_reply_to_status_id', 'in_reply_to_status_id_str', 'in_reply_to_user_id', 'in_reply_to_user_id_str', 'in_reply_to_screen_name', 'author', 'user', 'geo', 'coordinates', 'place', 'contributors', 'is_quote_status', 'retweet_count', 'favorite_count', 'favorited', 'retweeted', 'possibly_sensitive', 'possibly_sensitive_appealable', 'lang'])

In [282]:
with open('../data/tweet_json.txt', 'w') as json_txt:
    num_missing = 0
    for index, row in twit_ar_df[['tweet_id']].iterrows():
        try:
            json_str = json.dumps(api.get_status(row['tweet_id'], tweet_mode='extended')._json)
        except Exception:
            num_missing += 1
        json_txt.write(json_str + '\n')
    print("Download finished! Num missing is {0}".format(str(num_missing)))

Rate limit reached. Sleeping for: 748
Rate limit reached. Sleeping for: 746


Download finished! Num missing is 16


## Assess Data Quality

In [283]:
img_pred_df = pd.read_csv('../data/image-predictions.tsv', sep='\t')

In [284]:
img_pred_df

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,1,Rhodesian_ridgeback,0.408143,True,redbone,0.360687,True,miniature_pinscher,0.222752,True
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,1,miniature_pinscher,0.560311,True,Rottweiler,0.243682,True,Doberman,0.154629,True
5,666050758794694657,https://pbs.twimg.com/media/CT5Jof1WUAEuVxN.jpg,1,Bernese_mountain_dog,0.651137,True,English_springer,0.263788,True,Greater_Swiss_Mountain_dog,0.016199,True
6,666051853826850816,https://pbs.twimg.com/media/CT5KoJ1WoAAJash.jpg,1,box_turtle,0.933012,False,mud_turtle,0.045885,False,terrapin,0.017885,False
7,666055525042405380,https://pbs.twimg.com/media/CT5N9tpXIAAifs1.jpg,1,chow,0.692517,True,Tibetan_mastiff,0.058279,True,fur_coat,0.054449,False
8,666057090499244032,https://pbs.twimg.com/media/CT5PY90WoAAQGLo.jpg,1,shopping_cart,0.962465,False,shopping_basket,0.014594,False,golden_retriever,0.007959,True
9,666058600524156928,https://pbs.twimg.com/media/CT5Qw94XAAA_2dP.jpg,1,miniature_poodle,0.201493,True,komondor,0.192305,True,soft-coated_wheaten_terrier,0.082086,True


In [285]:
img_pred_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
tweet_id    2075 non-null int64
jpg_url     2075 non-null object
img_num     2075 non-null int64
p1          2075 non-null object
p1_conf     2075 non-null float64
p1_dog      2075 non-null bool
p2          2075 non-null object
p2_conf     2075 non-null float64
p2_dog      2075 non-null bool
p3          2075 non-null object
p3_conf     2075 non-null float64
p3_dog      2075 non-null bool
dtypes: bool(3), float64(3), int64(2), object(4)
memory usage: 152.1+ KB


In [286]:
twit_ar_df

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,None,None,None,None
3,891689557279858688,NaN,NaN,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891689557...,13,10,Darla,None,None,None,None
4,891327558926688256,NaN,NaN,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891327558...,12,10,Franklin,None,None,None,None
5,891087950875897856,NaN,NaN,2017-07-29 00:08:17 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Here we have a majestic great white breaching ...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891087950...,13,10,None,None,None,None,None
6,890971913173991426,NaN,NaN,2017-07-28 16:27:12 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Meet Jax. He enjoys ice cream so much he gets ...,NaN,NaN,NaN,"https://gofundme.com/ydvmve-surgery-for-jax,ht...",13,10,Jax,None,None,None,None
7,890729181411237888,NaN,NaN,2017-07-28 00:22:40 +0000,"<a href=""http://twitter.com/download/iphone"" r...",When you watch your owner call another dog a g...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/890729181...,13,10,None,None,None,None,None
8,890609185150312448,NaN,NaN,2017-07-27 16:25:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Zoey. She doesn't want to be one of th...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/890609185...,13,10,Zoey,None,None,None,None
9,890240255349198849,NaN,NaN,2017-07-26 15:59:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Cassie. She is a college pup. Studying...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/890240255...,14,10,Cassie,doggo,None,None,None


In [287]:
twit_ar_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
tweet_id                      2356 non-null int64
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2356 non-null object
source                        2356 non-null object
text                          2356 non-null object
retweeted_status_id           181 non-null float64
retweeted_status_user_id      181 non-null float64
retweeted_status_timestamp    181 non-null object
expanded_urls                 2297 non-null object
rating_numerator              2356 non-null int64
rating_denominator            2356 non-null int64
name                          2356 non-null object
doggo                         2356 non-null object
floofer                       2356 non-null object
pupper                        2356 non-null object
puppo                         2356 non-null object
dtypes: float64(4), int64(3), ob

In [288]:
twit_ar_df.loc[:,'source'].unique()

array(['<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>',
       '<a href="http://twitter.com" rel="nofollow">Twitter Web Client</a>',
       '<a href="http://vine.co" rel="nofollow">Vine - Make a Scene</a>',
       '<a href="https://about.twitter.com/products/tweetdeck" rel="nofollow">TweetDeck</a>'],
      dtype=object)

In [289]:
duplicated_in = twit_ar_df.loc[:,'name'].duplicated()
list(sorted(twit_ar_df[duplicated_in].loc[:,'name'].unique()))

['Abby',
 'Albert',
 'Albus',
 'Alfie',
 'Alice',
 'Anakin',
 'Archie',
 'Arnie',
 'Ash',
 'Aspen',
 'Astrid',
 'Atlas',
 'Atticus',
 'Ava',
 'Axel',
 'Bailey',
 'Baloo',
 'Balto',
 'Baxter',
 'Bear',
 'Beau',
 'Bell',
 'Bella',
 'Belle',
 'Benedict',
 'Benji',
 'Bentley',
 'Bernie',
 'Betty',
 'Bisquick',
 'Blitz',
 'Bo',
 'Bob',
 'Boomer',
 'Brad',
 'Brody',
 'Bruce',
 'Bubbles',
 'Buddy',
 'Bungalo',
 'Butter',
 'Calbert',
 'Cali',
 'Calvin',
 'Canela',
 'Carl',
 'Carly',
 'Cash',
 'Cassie',
 'CeCe',
 'Charles',
 'Charlie',
 'Chelsea',
 'Chester',
 'Chet',
 'Chip',
 'Chipson',
 'Chompsky',
 'Chuckles',
 'Churlie',
 'Clarence',
 'Clark',
 'Coco',
 'Cody',
 'Colby',
 'Cooper',
 'Coops',
 'Crystal',
 'Cupcake',
 'Curtis',
 'Daisy',
 'Dakota',
 'Dash',
 'Dave',
 'Davey',
 'Dawn',
 'Derek',
 'Dexter',
 'Django',
 'Doc',
 'Doug',
 'Duke',
 'Earl',
 'Eli',
 'Ellie',
 'Elliot',
 'Emmy',
 'Eve',
 'Finley',
 'Finn',
 'Fiona',
 'Fizz',
 'Flávio',
 'Frank',
 'Frankie',
 'Franklin',
 'Fred',
 'G

In [290]:
tweet_dicts = []
with open('../data/tweet_json.txt', 'r') as json_file:
    for line in json_file.readlines():
        tweet_dicts.append(json.loads(line))
tweet_json_df = pd.DataFrame(tweet_dicts)

In [291]:
tweet_json_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 32 columns):
contributors                     0 non-null object
coordinates                      0 non-null object
created_at                       2356 non-null object
display_text_range               2356 non-null object
entities                         2356 non-null object
extended_entities                2082 non-null object
favorite_count                   2356 non-null int64
favorited                        2356 non-null bool
full_text                        2356 non-null object
geo                              0 non-null object
id                               2356 non-null int64
id_str                           2356 non-null object
in_reply_to_screen_name          78 non-null object
in_reply_to_status_id            78 non-null float64
in_reply_to_status_id_str        78 non-null object
in_reply_to_user_id              78 non-null float64
in_reply_to_user_id_str          78 non-null obj

In [292]:
tweet_json_df

,contributors,coordinates,created_at,display_text_range,entities,extended_entities,favorite_count,favorited,full_text,geo,...,quoted_status,quoted_status_id,quoted_status_id_str,quoted_status_permalink,retweet_count,retweeted,retweeted_status,source,truncated,user
0,None,None,Tue Aug 01 16:23:56 +0000 2017,"[0, 85]","{'hashtags': [], 'symbols': [], 'user_mentions...","{'media': [{'id': 892420639486877696, 'id_str'...",38028,False,This is Phineas. He's a mystical boy. Only eve...,None,...,NaN,NaN,NaN,NaN,8317,False,NaN,"<a href=""http://twitter.com/download/iphone"" r...",False,"{'id': 4196983835, 'id_str': '4196983835', 'na..."
1,None,None,Tue Aug 01 00:17:27 +0000 2017,"[0, 138]","{'hashtags': [], 'symbols': [], 'user_mentions...","{'media': [{'id': 892177413194625024, 'id_str'...",32646,False,This is Tilly. She's just checking pup on you....,None,...,NaN,NaN,NaN,NaN,6143,False,NaN,"<a href=""http://twitter.com/download/iphone"" r...",False,"{'id': 4196983835, 'id_str': '4196983835', 'na..."
2,None,None,Mon Jul 31 00:18:03 +0000 2017,"[0, 121]","{'hashtags': [], 'symbols': [], 'user_mentions...","{'media': [{'id': 891815175371796480, 'id_str'...",24570,False,This is Archie. He is a rare Norwegian Pouncin...,None,...,NaN,NaN,NaN,NaN,4067,False,NaN,"<a href=""http://twitter.com/download/iphone"" r...",False,"{'id': 4196983835, 'id_str': '4196983835', 'na..."
3,None,None,Sun Jul 30 15:58:51 +0000 2017,"[0, 79]","{'hashtags': [], 'symbols': [], 'user_mentions...","{'media': [{'id': 891689552724799489, 'id_str'...",41381,False,This is Darla. She commenced a snooze mid meal...,None,...,NaN,NaN,NaN,NaN,8461,False,NaN,"<a href=""http://twitter.com/download/iphone"" r...",False,"{'id': 4196983835, 'id_str': '4196983835', 'na..."
4,None,None,Sat Jul 29 16:00:24 +0000 2017,"[0, 138]","{'hashtags': [{'text': 'BarkWeek', 'indices': ...","{'media': [{'id': 891327551943041024, 'id_str'...",39556,False,This is Franklin. He would like you to stop ca...,None,...,NaN,NaN,NaN,NaN,9159,False,NaN,"<a href=""http://twitter.com/download/iphone"" r...",False,"{'id': 4196983835, 'id_str': '4196983835', 'na..."
5,None,None,Sat Jul 29 00:08:17 +0000 2017,"[0, 138]","{'hashtags': [{'text': 'BarkWeek', 'indices': ...","{'media': [{'id': 891087942176911360, 'id_str'...",19866,False,Here we have a majestic great white breaching ...,None,...,NaN,NaN,NaN,NaN,3049,False,NaN,"<a href=""http://twitter.com/download/iphone"" r...",False,"{'id': 4196983835, 'id_str': '4196983835', 'na..."
6,None,None,Fri Jul 28 16:27:12 +0000 2017,"[0, 140]","{'hashtags': [], 'symbols': [], 'user_mentions...","{'media': [{'id': 890971906207338496, 'id_str'...",11615,False,Meet Jax. He enjoys ice cream so much he gets ...,None,...,NaN,NaN,NaN,NaN,2018,False,NaN,"<a href=""http://twitter.com/download/iphone"" r...",False,"{'id': 4196983835, 'id_str': '4196983835', 'na..."
7,None,None,Fri Jul 28 00:22:40 +0000 2017,"[0, 118]","{'hashtags': [], 'symbols': [], 'user_mentions...","{'media': [{'id': 890729118844600320, 'id_str'...",64171,False,When you watch your owner call another dog a g...,None,...,NaN,NaN,NaN,NaN,18450,False,NaN,"<a href=""http://twitter.com/download/iphone"" r...",False,"{'id': 4196983835, 'id_str': '4196983835', 'na..."
8,None,None,Thu Jul 27 16:25:51 +0000 2017,"[0, 122]","{'hashtags': [{'text': 'BarkWeek', 'indices': ...","{'media': [{'id': 890609177319665665, 'id_str'...",27299,False,This is Zoey. She doesn't want to be one of th...,None,...,NaN,NaN,NaN,NaN,4186,False,NaN,"<a href=""http://twitter.com/download/iphone"" r...",False,"{'id': 4196983835, 'id_str': '4196983835', 'na..."
9,None,None,Wed Jul 26 15:59:51 +0000 2017,"[0, 133]","{'hashtags': [], 'symbols': [], 'user_mentions...","{'media': [{'id': 890240245463175168, 'id_str'...",31308,False,This is Cassie. She is a college pup. Studying...,None,...,NaN,NaN,NaN,NaN,7215,False,NaN,"<a href=""http://twitter.com/download/iphone"" r...",False,"{'id': 4196983835, 'id_str': '4196983835', 'na..."


### Cleanliness Issues

- There are missing dog names (None) in the "name" column of the Tweet archive DataFrame
- The Twitter data source is still in raw HTML and needs to be converted
- Some dog names don't make sense ('a', 'actually', 'given')
- The breed names in the image prediction csv have underscores and are capitalized wrong
- Some of the ratings are blatantly wrong with some numerators not above 10 and one having a denominator of 2
- Some expanded URLs in the Tweet archive data frame are missing
- In the image predictions, there are records that are clearly not dogs that need to be removed.
- The additional downloaded data from Twitter doesn't have the same record count as the archive, therefore there is missing data.

### Tidyness Issues

- The twitter archive and newly downloaded data need to be joined.
- The columns **floofer**, **doggo**, **pupper**, and **puppo** should be gathered into one categorical column.

## Clean

### Quality Cleaning Plan

- Remove all dog names named "none" and erroneous dog names that don't make sense.
- Extract data source string info from extra HTML
- Remove underscores in breed names and switch capitalization to title case
- Remove all records with wrong ratings
- Keep as they are missing records that can't be changed
- Remove all non-dog records from empty data frame
- Remove repeat records from the data gathered from the Twitter API
- Change id name column in Twitter API data frame so that it can be joined to the archive data frame

### Tidiness Cleaning Plan

- Join the twitter archive and downloaded twitter data into one data frame
- Gather all of the dog category columns into one categorical column.

### Create Copies of Gathered Datasets

In [293]:
tweet_ar_clean = twit_ar_df.copy()
img_pred_df_clean = img_pred_df.copy()
tweet_json_clean = tweet_json_df.copy()

### Clean Quality Problems

#### Extract Source Information from HTML

In [294]:
html_strp = tweet_ar_clean.loc[:,'source'].str.extract('<.+>(.+)<.+>').iloc[:,0]
tweet_ar_clean.loc[:, 'source'] = html_strp.str.replace(' - Make a Scene', '')

In [295]:
assert list(tweet_ar_clean.loc[:, 'source'].unique()) == \
['Twitter for iPhone', 'Twitter Web Client', 'Vine', 'TweetDeck']

#### Remove All "None" Dog Names

In [296]:
tweet_ar_clean = tweet_ar_clean.query("name != 'None'").reset_index()

In [297]:
assert 'None' not in tweet_ar_clean.loc[:,'name'].unique()

#### Remove All Incorrect Dog Names

In [298]:
tweet_ar_clean = tweet_ar_clean.loc[~tweet_ar_clean['name'].str.islower(),:]

In [299]:
assert tweet_ar_clean['name'].str.islower().any() == False

#### Replace Underscores with Spaces in Image Recommends

In [300]:
for colname in ('p1', 'p2', 'p3'):
    img_pred_df_clean[colname] = img_pred_df_clean[colname].str.replace('_', ' ').str.title()

#### Change Recommendations to Title Case and test to See if the Two columns are clean

In [301]:
for colname in ('p1', 'p2', 'p3'):
    assert img_pred_df_clean[colname].str.istitle().all() == True
    uscore_array = img_pred_df_clean['p1'].str.find('_').values > 0
    assert uscore_array.any() == False

#### Remove Ratings that Have a Denominator Greater Than it's Numerator

In [302]:
tweet_ar_clean = tweet_ar_clean.query('rating_numerator > rating_denominator')

In [303]:
is_greater = tweet_ar_clean['rating_numerator'] > tweet_ar_clean['rating_denominator']
assert is_greater.values.all() == True

#### Remove All Records that Have Denominators Not Equal to 10

In [304]:
tweet_ar_clean = tweet_ar_clean.query('rating_denominator == 10')

In [305]:
is_two = tweet_ar_clean['rating_denominator']  == 10
assert is_two.values.all() == True

#### Remove All Non-Dog Recommendations from Recommendation DataFrame

In [306]:
img_pred_df_clean = img_pred_df_clean.query("p1_dog == True & p2_dog == True & p3_dog == True")

In [307]:
assert img_pred_df_clean['p1_dog'].values.all() == True
assert img_pred_df_clean['p2_dog'].values.all() == True
assert img_pred_df_clean['p3_dog'].values.all() == True

#### Rename ID column to the Correct name that will be Joined

In [308]:
tweet_json_clean = tweet_json_clean.rename(columns={'id': 'tweet_id'})

In [309]:
assert 'tweet_id' in list(tweet_json_clean.columns.values)

#### Select The Only Needed Columns from the Twitter Data Gathered from API

In [310]:
tweet_json_clean = tweet_json_clean[['tweet_id', 'favorite_count', 'retweet_count']]

In [311]:
assert list(tweet_json_clean.columns.values) == ['tweet_id', 'favorite_count', 'retweet_count']

### Clean Tidyness Problems

#### Convert Stage Columns into a Single Categorical Variable

In [312]:
id_cols = [
    'tweet_id',                      
    'in_reply_to_status_id',         
    'in_reply_to_user_id',           
    'timestamp',                     
    'source',                        
    'text',                          
    'retweeted_status_id',        
    'retweeted_status_user_id',      
    'retweeted_status_timestamp',    
    'expanded_urls',                 
    'rating_numerator',              
    'rating_denominator',            
    'name'
]

tweet_ar_clean = pd.melt(
    tweet_ar_clean, id_vars = id_cols, value_vars = ['floofer', 'puppo', 'doggo', 'pupper']
    ).query("value != 'None'").drop('variable', axis=1).rename(columns={'value': 'stage'})

In [313]:
assert 'stage' in tweet_ar_clean.columns
assert list(tweet_ar_clean['stage'].unique()) == ['floofer', 'puppo', 'doggo', 'pupper']

#### Merge the Tweet Archive and API Data Together based on Tweet ID

In [314]:
tweet_ar_clean = pd.merge(tweet_ar_clean, tweet_json_clean, on="tweet_id")

In [315]:
assert 'favorite_count' in list(tweet_ar_clean.columns.values)
assert 'retweet_count' in list(tweet_ar_clean.columns.values)

### Output Clean Data Into Files

In [316]:
tweet_ar_clean.to_csv('../data/twitter_archive_master.csv', index=False)

In [317]:
img_pred_df_clean.to_csv('../data/img_preds_master.csv', index=False)